In [ ]:
#%pip install apify_client

In [ ]:
# Dependancies
import pandas as pd
from apify_client import ApifyClient
import time
import os
from dotenv import load_dotenv

In [9]:
# Read post data
data = pd.read_excel("post.xlsx")
data.head(3)

,media/0/__isMedia,media/0/__typename,media/0/accent_color,media/0/animated_image_caption,media/0/audio_availability,media/0/audio_settings,media/0/audio_user_preferred_language,media/0/autoplay_gating_result,media/0/breakingStatus,media/0/broadcast_id,...,media/2/video_player_shaka_performance_logger_init2/per_session_sampling_rate,media/2/video_player_shaka_performance_logger_should_sample,media/2/video_status_type,media/2/viewer_autoplay_setting,media/2/width,url,text,likes,comments,shares
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/posts...,"«Je ne perds jamais, soit je gagne, soit j'app...",1811,56,18
1,NaN,Video,NaN,NaN,AVAILABLE,NaN,en,all_page_organic_allowed,False,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/video...,Triste.,2521,1111,530
2,Photo,Photo,FF000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/ibrahimamaigabf/posts...,NaN,2457,441,95


In [12]:
# Choosing important columns
data = data[["url", "text", "likes", "comments", "shares"]]
data.head(3)

,url,text,likes,comments,shares
0,https://www.facebook.com/ibrahimamaigabf/posts...,"«Je ne perds jamais, soit je gagne, soit j'app...",1811,56,18
1,https://www.facebook.com/ibrahimamaigabf/video...,Triste.,2521,1111,530
2,https://www.facebook.com/ibrahimamaigabf/posts...,NaN,2457,441,95


In [13]:
# Collect posts urls
urls = list(data["url"])

In [14]:
urls

['https://www.facebook.com/ibrahimamaigabf/posts/pfbid0LiK3A8uop129UP42AG9sa7trg3NHobGyn2ZVaPheH5QzZ9t4pyzyvX7dfu2rGj5fl',
 'https://www.facebook.com/ibrahimamaigabf/videos/1099971173944828/',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0rTqBUHiv3vZDBuaVzYPha6R8brJPzYnhkY1eDcSHUUi8PuApfQrHgcgCeTiQfEPBl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02b7mW9KjYQYeFr5vzcbd5xwgV9ntrp6yTo5w1zRYBfpwKypYpEjojPcAHd9PjsFELl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02dYyc7Nc8nicHUadmFmf5fCaYZ1MtLd3K4YguWJ3xojp9Gr8FXVUquG56mVEwynUwl',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0ckte351QT6NqQecc8KtHipcXmQ5dW6mZcKy4nVB6SNubnFupwpRafLkv861dExhel',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid0snAoVBvUph1CGj8NwbJnD1sqXezP8T5NBcsGLVwcYhU2w62pugFJAY2VJK4S5uUTl',
 'https://www.facebook.com/ibrahimamaigabf/videos/1230999364113693/',
 'https://www.facebook.com/ibrahimamaigabf/posts/pfbid02t1VWih6pehgvNg9gcrTpYYkn4i4CNoy4TvUu5yEi72y7k85DT5Sbspe3oEfoqGWU

### Comments Scraping

In [ ]:
# Config
API_TOKEN = os.getenv("APIFY_API_TOKEN")
ACTOR_ID = os.getenv("ACTOR_ID")
OUTPUT_CSV = "facebook_comments.csv"  # Fichier de sortie
MAX_RESULTS = 100                 # Nombre max de commentaires par post
INCLUDE_NESTED = True           # Inclure les réponses aux commentaires

In [20]:
client = ApifyClient(API_TOKEN)

In [21]:
all_comments = []

for url in urls:
    print(f"📥 Récupération des commentaires pour : {url}")

    # Préparer l'entrée pour l'Actor
    run_input = {
        "startUrls": [{"url": url}],
        "resultsLimit": MAX_RESULTS,
        "includeNestedComments": INCLUDE_NESTED,
        "viewOption": "RANKED_UNFILTERED"
    }

    # Lancer l'Actor
    try:
        run = client.actor(ACTOR_ID).call(run_input=run_input)
    except Exception as e:
        print(f"⚠️ Erreur pour {url}: {e}")
        continue

    # Parcourir les résultats
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        comment_data = {
            "post_url": url,
            "comment_id": item.get("id"),
            "comment_text": item.get("text"),
            "comment_author": item.get("author", {}).get("name"),
            "comment_time": item.get("timestamp"),
            "comment_reactions": item.get("reactionsCount"),
            "reply_count": item.get("replyCount"),
        }
        all_comments.append(comment_data)

    # Pause légère pour éviter de saturer l'API
    time.sleep(3)

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0LiK3A8uop129UP42AG9sa7trg3NHobGyn2ZVaPheH5QzZ9t4pyzyvX7dfu2rGj5fl


[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> Status: RUNNING, Message: 
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:11.315Z ACTOR: Pulling container image of build ZWXhjwBtDa2tm659a from registry.
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:11.317Z ACTOR: Creating container.
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:11.724Z ACTOR: Starting container.
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:12.939Z INFO  System info {"apifyVersion":"3.4.3","apifyClientVersion":"2.12.5","crawleeVersion":"3.13.7","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:13.078Z INFO  Results Limit 50
[apify.facebook-comments-scraper runId:muonLz4ZCfkyPMlTh] -> 2025-10-11T13:56:13.083Z INFO  CheerioCrawler: Using the old RequestQueue implementation without request locking.
[apify.facebook-

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/videos/1099971173944828/


[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> Status: RUNNING, Message: 
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:44.892Z ACTOR: Pulling container image of build ZWXhjwBtDa2tm659a from registry.
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:44.894Z ACTOR: Creating container.
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:44.947Z ACTOR: Starting container.
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:46.026Z INFO  System info {"apifyVersion":"3.4.3","apifyClientVersion":"2.12.5","crawleeVersion":"3.13.7","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:46.213Z INFO  Results Limit 50
[apify.facebook-comments-scraper runId:RG7NuISo9ZwQguX7K] -> 2025-10-11T13:56:46.221Z INFO  CheerioCrawler: Using the old RequestQueue implementation without request locking.
[apify.facebook-

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0rTqBUHiv3vZDBuaVzYPha6R8brJPzYnhkY1eDcSHUUi8PuApfQrHgcgCeTiQfEPBl


...

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid02GkkYU9CUnfbf4vwwHHU6PY5hi7zqC29KXKbRauL3uYEDtR2mZHKUoRtQnJVboVLLl


[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> Status: RUNNING, Message: 
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:38.106Z ACTOR: Pulling container image of build ZWXhjwBtDa2tm659a from registry.
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:38.108Z ACTOR: Creating container.
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:38.161Z ACTOR: Starting container.
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:39.651Z INFO  System info {"apifyVersion":"3.4.3","apifyClientVersion":"2.12.5","crawleeVersion":"3.13.7","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:39.785Z INFO  Results Limit 50
[apify.facebook-comments-scraper runId:lVYIGCVslpKmEgDXx] -> 2025-10-11T14:10:39.790Z INFO  CheerioCrawler: Using the old RequestQueue implementation without request locking.
[apify.facebook-

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid02AdVSCs3coUnKUE9hR78ggz5Vdu3zHzn6iVgWmfJutxDQAhRqSZawQA8yS9UbSQePl


[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> Status: RUNNING, Message: Crawled 3/4 pages, 0 failed requests, desired concurrency 10.
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:07.161Z ACTOR: Pulling container image of build ZWXhjwBtDa2tm659a from registry.
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:07.167Z ACTOR: Creating container.
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:07.201Z ACTOR: Starting container.
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:08.640Z INFO  System info {"apifyVersion":"3.4.3","apifyClientVersion":"2.12.5","crawleeVersion":"3.13.7","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:08.811Z INFO  Results Limit 50
[apify.facebook-comments-scraper runId:teyx5iRVpY79Ibhsz] -> 2025-10-11T14:11:08.818Z INFO  CheerioCrawler: Using the old RequestQ

📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0VZbiLgAZ9agwcXUM9Yxz5umY3YJz5z8JVqofjrMikr5jrCNZqEYTKy3EytjanNKml
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/posts/pfbid0VZbiLgAZ9agwcXUM9Yxz5umY3YJz5z8JVqofjrMikr5jrCNZqEYTKy3EytjanNKml: Monthly usage hard limit exceeded
📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid02AW4iB5iwfwD5EPN2YdFW8PESuWTH8UBpCn7HJB5bYrxMXCAR62zg9CkLa8EG7CUEl
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/posts/pfbid02AW4iB5iwfwD5EPN2YdFW8PESuWTH8UBpCn7HJB5bYrxMXCAR62zg9CkLa8EG7CUEl: Monthly usage hard limit exceeded
📥 Récupération des commentaires pour : https://www.facebook.com/ibrahimamaigabf/posts/pfbid0WMFQMPeMD55ZV8S6WFVJe9utJpM6zrATDRw1c3VZnnr3B2mSRtWVjvHVXrXqCsaMl
⚠️ Erreur pour https://www.facebook.com/ibrahimamaigabf/posts/pfbid0WMFQMPeMD55ZV8S6WFVJe9utJpM6zrATDRw1c3VZnnr3B2mSRtWVjvHVXrXqCsaMl: Monthly usage hard limit exceeded
📥 Récupération de

In [22]:
if all_comments:
    comments_df = pd.DataFrame(all_comments)
    comments_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
    print(f"\n✅ {len(all_comments)} commentaires enregistrés dans '{OUTPUT_CSV}'")
else:
    print("\n⚠️ Aucun commentaire collecté.")


✅ 965 commentaires enregistrés dans 'facebook_comments.csv'


In [23]:
comments = pd.read_csv("facebook_comments.csv")
comments.head(3)

,post_url,comment_id,comment_text,comment_author,comment_time,comment_reactions,reply_count
0,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfMTEyNzIyNDYyND...,Maiga il est temps de passer a l action car no...,NaN,NaN,NaN,NaN
1,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfODU3NTYwMzQxOD...,NaN,NaN,NaN,NaN,NaN
2,https://www.facebook.com/ibrahimamaigabf/posts...,Y29tbWVudDo0NjIwNTA5MDU5NzM3MTFfMTI4ODEzNTY3MT...,Affirmative 🙏,NaN,NaN,NaN,NaN


In [24]:
comments = comments[["post_url", "comment_text"]]
comments.columns = ["url", "comment_text"]
comments.head(3)

,url,comment_text
0,https://www.facebook.com/ibrahimamaigabf/posts...,Maiga il est temps de passer a l action car no...
1,https://www.facebook.com/ibrahimamaigabf/posts...,NaN
2,https://www.facebook.com/ibrahimamaigabf/posts...,Affirmative 🙏


In [ ]:
# merge data and comments